In [4]:
import yfinance as yf
import numpy as np
import pandas as pd

In [5]:
tickers = ["AAPL", "GOOGL", "TSLA"]
start_date = "2023-02-07"
end_date = "2024-02-07"

data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%***********************]  3 of 3 completed


In [6]:
close_data = data["Close"]
close_data.tail()

Ticker,AAPL,GOOGL,TSLA
Date,,,
2024-01-31,183.501022,139.593964,187.289993
2024-02-01,185.949036,140.650146,188.860001
2024-02-02,184.943954,141.865738,187.910004
2024-02-05,186.765015,143.161026,181.059998
2024-02-06,188.377136,143.579514,185.100006


In [7]:
log_returns = np.log(close_data / close_data.shift(1))
log_returns.tail()

Ticker,AAPL,GOOGL,TSLA
Date,,,
2024-01-31,-0.019547,-0.077965,-0.022699
2024-02-01,0.013252,0.007538,0.008348
2024-02-02,-0.005420,0.008606,-0.005043
2024-02-05,0.009798,0.009089,-0.037135
2024-02-06,0.008595,0.002919,0.022068


In [8]:
expected_daily_returns = log_returns.mean()
daily_std_dev = log_returns.std()

In [9]:
pd.DataFrame({
    "Ticker": tickers,
    "Expected Daily Return": expected_daily_returns.values,
    "Daily Standard Deviation": daily_std_dev.values
})

,Ticker,Expected Daily Return,Daily Standard Deviation
0,AAPL,0.000831,0.012344
1,GOOGL,0.001167,0.018351
2,TSLA,-0.000245,0.031385


### Portfolio Expected Return and Risk

In [10]:
weights_column_matrix = pd.DataFrame([0.4, 0.35, 0.25], index=tickers, columns=["Weight"])


expected_portfolio_return = np.sum(weights_column_matrix["Weight"] * expected_daily_returns)
print(f"Expected Portfolio Return (Daily): {expected_portfolio_return * 100:.2f}%")

Expected Portfolio Return (Daily): 0.07%


In [11]:
portfolio_std_dev = np.sqrt(np.dot(weights_column_matrix.T, np.dot(log_returns.cov(), weights_column_matrix)))
print(f"Portfolio Risk: {portfolio_std_dev[0, 0] * 100:.2f}%")

Portfolio Risk: 1.46%


### VaR Calculation

In [17]:
from scipy.stats import norm

confidence_95 = 0.95  # 95% confidence
confidence_99 = 0.99  # 99% confidence

z_95 = norm.ppf(1 - confidence_95)  # Left tail (5%)
z_99 = norm.ppf(1 - confidence_99)  # Left tail (1%)

var_95 = expected_portfolio_return + z_95 * portfolio_std_dev
var_99 = expected_portfolio_return + z_99 * portfolio_std_dev

# print(f"95% VaR: {var_95[0, 0] * 100:.2f}%")
print(f"95% probability that the portfolio will not lose more than {var_95[0, 0] * 100:.2f}%")
# print(f"99% VaR: {var_99[0, 0] * 100:.2f}%")
print(f"99% probability that the portfolio will not lose more than {var_99[0, 0] * 100:.2f}%")

95% probability that the portfolio will not lose more than -2.34%
99% probability that the portfolio will not lose more than -3.34%
